In [9]:
import requests as r
from bs4 import BeautifulSoup
from collections import Counter
import time
import threading
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed


HOST = 'https://www.104.com.tw'
HEADER = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.96 Safari/537.36'}

# global variable
fail_pages = []   
fail_links = []
thread_list=[]
alinks = []
total_links = 0
wc = Counter()
lock = threading.Lock()
link_path = '../data/link_104.csv'

# Get all links from 150 pages
for i in range(1,150):
    url =  HOST + '/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=1&keyword=%E8%BB%9F%E9%AB%94%E5%B7%A5%E7%A8%8B%E5%B8%AB&order=1&asc=0&page={}'.format(i)
    th = threading.Thread(target=get_pagelinks, args=(url, lock, i, link_path))
    th.start()
    time.sleep(1)
    thread_list.append(th)
else: 
    print('[Debug] All threads are started')

# Join all threads
for thread in thread_list:
    thread.join()
    
print('Total_links {:04}'.format(total_links))


# Get all content from alllinks
with ThreadPoolExecutor(max_workers=20) as executor:
    for idx, link in enumerate(alinks):
        executor.submit(grab_content, link, lock, idx) 
        
# Wirte results to file
with open('104.csv', 'w') as fw2:
    for lang, counts in wc.most_common():
        fw2.write('{},{}\n'.format(lang,counts))

Cannot retrieve links from page 1 Retry: 1 
Cannot retrieve links from page 2 Retry: 1 
Cannot retrieve links from page 1 Retry: 2 
Cannot retrieve links from page 3 Retry: 1 
Cannot retrieve links from page 2 Retry: 2 
Cannot retrieve links from page 1 Retry: 3 
Cannot retrieve links from page 4 Retry: 1 
Cannot retrieve links from page 3 Retry: 2 
Cannot retrieve links from page 2 Retry: 3 
Cannot retrieve links from page 1 Retry: 4 
Cannot retrieve links from page 5 Retry: 1 
Cannot retrieve links from page 4 Retry: 2 
Cannot retrieve links from page 3 Retry: 3 
Cannot retrieve links from page 6 Retry: 1 
Cannot retrieve links from page 2 Retry: 4 
Cannot retrieve links from page 1 Retry: 5 
Cannot retrieve links from page 5 Retry: 2 


KeyboardInterrupt: 

In [1]:
def grab_content(url,lock,idx): #一個用來分析內頁的function
    global total_links
    global HEADER
    try:
        count = 5
        while count:
            res2 = r.get(url, headers=HEADER)
            if res2.status_code == 200:
                soup = BeautifulSoup(res2.text, 'lxml')
                lans = soup.select('.tool')[0].select('a')  #lans  is  list
                global wc
                with lock:      
                    for lan in lans: 
                        if lan.text in wc:  
                            wc[lan.text] += 1
                        else:
                            wc[lan.text] = 1
                if idx % 100 == 0:
                    print('{} links remain'.format(total_links - 50))
            else:
                count -= 1
                time.sleep(0.5)
        time.sleep(0.5)
        if count == 0:
            fail_links.append(url)
        return 
    except Exception as e:
        print(e)


In [6]:
def get_pagelinks(url, lock, i, link_path):  #找所有連結
    
    global HEADER
    global alinks
    global total_links
    global HOST
    
    page_links = [] 
    try:
        count = 5             # retry 5 times
        while count:
            res = r.get(url, headers=HEADER)
            if res.status_code == 200:
                
                soup = BeautifulSoup(res.text, 'lxml')
                links = soup.select(".j_cont > ul > li  > div > a")
                
                if len(links) != 0: 
                    for link in links:
                        if("jobno" in link['href']):  
                             page_links.append(HOST + link['href']) #href put in innerURL function           
                    print('Get {} links from page {}'.format(len(page_links), i))

                    with open(link_path, 'a') as fw:
                        with lock:
                            total_links += len(page_links)
                            alinks += page_links
                            fw.write('\n'.join(page_links))
                            fw.write('\n')
                        break
                else:
                    print ('Cannot retrieve links from page {} Retry: {} '.format(i, 6 - count))
                    count -= 1
                    time.sleep(1)
            else:
                count -= 1
                time.sleep(1)
        if count == 0:
            fail_pages.append(url)
            print('[Error] Page {:2} failed'.format(i))
    except Exception as e:
        print(e)


In [ ]:
# abandoned
def grab_threadpool(alinks, lock):
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(grab_content, link, lock) for link in alinks]
        print('page is finish')

In [ ]:
# abandoned 
class GrabLinkThread (threading.Thread): # new MyClass extends Thread  
    
    def __init__(self, url, lock):   # __init__  is a constructor    #args傳入
        super(GrabLinkThread, self).__init__()  #super()  傳到父類別的建構子
        self.url = url
        self.lock = lock
    
    def run(self):   #等於java中的run  overwrite  
        pageURL(url, self.lock)